In [1]:
import os
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten, Input, Activation, Add, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Convolution3D, MaxPooling3D, GlobalAveragePooling3D, Lambda, UpSampling3D, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from keras.losses import mse
%matplotlib inline

Using TensorFlow backend.


In [8]:
is_channels_first = False
buf = np.load('./data/resnet3d_data_channel_last_updown_and_wipe_frame-3_stride123_gray_100x100_20191016.npz')

X = buf["X"].astype("f2")
Y = buf["Y"]
del buf

Y = keras.utils.to_categorical(Y, 2, dtype="f2").astype("f2")
X_train, X_val,  Y_train,  Y_val = train_test_split(X, Y, train_size=0.8, shuffle=True)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, train_size=0.8, shuffle=True)

In [3]:
class VAE:
    def __init__(self, frame = 3, height = 100, width = 100, channel = 1, latent_dim = 100):
        self.frame = frame
        self.height = height
        self.width = width
        self.channel = channel
        self.latent_dim = latent_dim
        self.input = Input(shape=(self.frame, self.height, self.width, self.channel))
        self.latent_inputs = Input(shape=(self.latent_dim,))        

    def Build_Encoder(self):
        x = Convolution3D(64, (3,3,3),  kernel_initializer='he_normal', padding='same', activation="relu")(self.input)
        x = MaxPooling3D((1,2,2), padding="same")(x)        

        x = Convolution3D(128, (3,3,3),  kernel_initializer='he_normal', padding='same', activation="relu")(x)
        x = MaxPooling3D((3,2,2), padding="same")(x)        
        self.shape = K.int_shape(x)

        x = Flatten()(x)
        z_mean = Dense(self.latent_dim, activation='linear')(x)
        z_log_var = Dense(self.latent_dim, activation='linear')(x)
        z = Lambda(self.Sampling, output_shape=(self.latent_dim,), name='z')([z_mean, z_log_var])
        return Model(self.input, [z_mean, z_log_var, z])
    
    def Build_Decoder(self):
        x = Dense(self.shape[1] * self.shape[2] * self.shape[3] * self.shape[4], activation='relu')(self.latent_inputs)
        x = Reshape((self.shape[1], self.shape[2], self.shape[3], self.shape[4]))(x)

        x = UpSampling3D((3, 2, 2))(x)
        x = Convolution3D(128, (3,3,3), activation='relu', padding='same')(x)
        x = UpSampling3D((1, 2, 2))(x)
        x = Convolution3D(64, (3,3,3), activation='relu', padding='same')(x)
        decoded = Convolution3D(1, (3,3,3), activation='relu', padding='same')(x)
        return Model(self.latent_inputs, decoded)        
    
    def Sampling(self, args):
        """
        reparameterization trick
        instead of sampling from Q(z|X), sample eps = N(0,I)
        z = z_mean + sqrt(var)*eps
        """
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean=0 and std=1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    def Build(self):
        self.encoder = self.Build_Encoder()
        self.decoder = self.Build_Decoder()
        z_mean, z_log_var, z = self.encoder(self.input)
        outputs = self.decoder(z)
        self.vae = Model(self.input, outputs)

        reconstruction_loss = mse(K.flatten(self.input), K.flatten(outputs))

        reconstruction_loss *= self.shape[1] * self.shape[2] * self.shape[3]
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        vae_loss = K.mean(reconstruction_loss + kl_loss)
        self.vae.add_loss(vae_loss)
        self.vae.compile(optimizer="adam")
        
    def fit(self, X_train, X_test, batch_size, epoch):
        self.vae.fit(X_train, batch_size = batch_size, epochs = epoch, validation_data=(X_test, None))        

In [4]:
vae = VAE()

In [5]:
vae.Build()

In [10]:
vae.fit(X_train, X_test, batch_size=32, epoch=2)

Train on 1574 samples, validate on 394 samples
Epoch 1/2
1574/1574 [==============================] - 92s 58ms/sample - loss: 86.4139


NotFoundError: 2 root error(s) found.
  (0) Not found:  Resource AnonymousIterator/AnonymousIterator6/class tensorflow::data::IteratorResource does not exist.
	 [[node IteratorGetNext (defined at C:\Users\fan2tamo\Anaconda3\envs\work\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
	 [[IteratorGetNext/_9]]
  (1) Not found:  Resource AnonymousIterator/AnonymousIterator6/class tensorflow::data::IteratorResource does not exist.
	 [[node IteratorGetNext (defined at C:\Users\fan2tamo\Anaconda3\envs\work\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_distributed_function_5902]

Function call stack:
distributed_function -> distributed_function
